In [7]:
### import libraries

import pandas as pd
import igraph as ig
import numpy as np
import networkx as nx
from auxiliar_functions.community_detection import sub_community_detection, add_type_commts
import matplotlib.pyplot as plt

In [8]:
str_dataset = "IoT"
str_distri = "Nr"

pos_bip_network = ig.read("../data/"+str_dataset+"/binet-"+str_dataset+"-"+str_distri+".gml")
print(pos_bip_network.summary())
print("|U|=", len(pos_bip_network.vs.select(type=0)),"|R|=", len(pos_bip_network.vs.select(type=1)), "\n" )

neg_bip_network = ig.read("../data/"+str_dataset+"/binet-"+str_dataset+"-"+str_distri+"-NEG.gml")
print(neg_bip_network.summary())
print("|U|=", len(neg_bip_network.vs.select(type=0)),"|R|=", len(neg_bip_network.vs.select(type=1)) )

IGRAPH UNWT 926 45608 -- 
+ attr: id (v), name (v), ranking (v), type (v), weight (e)
|U|= 116 |R|= 810 

IGRAPH UNWT 893 37710 -- 
+ attr: id (v), name (v), ranking (v), type (v), weight (e)
|U|= 108 |R|= 785


# Proyección

In [46]:
def search_resource_in_commty(resource_id, commty):
    """ Search the resource in the commty. If the resource is in the commty
    returns True."""    
    for node in commty.vs(): # Por cada nodo en la comunidad        
        if resource_id in node["rsrcs"]: # Check if the user accesed to resource
            return True
        break
    return False

def get_edge_weight_2(i_node, j_node, bi_graph):
    """
    Compute the weight of an edge between i and j nodes.

    Args:
        i_node (networkX node): i node.
        j_node (networkX node): j node.

    Returns:
        weight (float): The weight between nodes.

    Raises:
        TypeError: if there are not an intersection
    """
    common_neighbors = set(i_node.neighbors()).intersection(j_node.neighbors())

    if not common_neighbors:
        raise ValueError("There are no common neighbors.")

    # Calculate intersection between two previous sets
    weighted_sum = sum(bi_graph[i_node,neighbor] for neighbor in common_neighbors)
    weighted_sum += sum(bi_graph[j_node,neighbor] for neighbor in common_neighbors)
    
    i_rank = sum(bi_graph[i_node, target] for target in i_node.neighbors())

    j_rank = sum(bi_graph[j_node, target] for target in j_node.neighbors())

    weight = (weighted_sum*weighted_sum) / (i_rank*j_rank)

    return weight

def bipartite_projection_2(biparte_network, node_type=0):
    """
    Generate a monopartite network from bipartite network.

    Parameters:
        bipartite_network (igraph Graph): The bipartie network.
        node_type (int): The set of nodes of the monopartite network.

    Returns:
        Graph (iGraph): The monopartite (projected) network.

    Raises:
        Some
    """

    # Check if the bipartite network is a bipartite network:
    # if not biparte_network.is_bipartite():
    #     raise TypeError("The ARBN is not bipartite")

    # networkX object (more easy to buil)
    g = nx.Graph()

    # All opposite node set
    opposite_nodes = biparte_network.vs.select(type=1)

    # Check for every node the same type
    for X_node in opposite_nodes:
        # Select all neighbors of the X_node
        neighbordhood = X_node.neighbors()

        for Y_node_i in neighbordhood:
            for Y_node_j in neighbordhood:
                # Ceck if both nodes are the same
                if Y_node_i['name'] != Y_node_j['name']:
                    # If there is no an edge generate
                    if not g.has_edge(Y_node_i['name'], Y_node_j['name']):
                        weight_ = get_edge_weight_2(Y_node_i, Y_node_j, biparte_network)
                        # print("Peso: ", Y_node_i['name'], "-", Y_node_j['name'], " => ", weight_)
                        g.add_edge(Y_node_i["name"], Y_node_j["name"],
                                   weight=weight_)

    # Convert from networkX graph to igraph graph
    g = ig.Graph.from_networkx(g)
    g.vs["name"] = g.vs["_nx_name"]
    del g.vs["_nx_name"]

    for u_nodes in g.vs:
        rsrcs = biparte_network.vs.find(name=u_nodes["name"]).neighbors()
        rsrcs = [r_node["name"] for r_node in rsrcs]
        u_nodes["rsrcs"] = rsrcs

    print("User Network builded!")
    print(g.summary())
    return g

def bipartite_projection_rs(biparte_network):
    """
    Generate a monopartite network from bipartite network.

    Parameters:
        bipartite_network (igraph Graph): The bipartie network.
        node_type (int): The set of nodes of the monopartite network.

    Returns:
        Graph (iGraph): The monopartite (projected) network.

    Raises:
        Some
    """

    # Check if the bipartite network is a bipartite network:
    # if not biparte_network.is_bipartite():
    #     raise TypeError("The ARBN is not bipartite")

    # networkX object (more easy to buil)
    g = nx.Graph()

    # All opposite node set
    opposite_nodes = biparte_network.vs.select(type=0)

    # Check for every node the same type
    for X_node in opposite_nodes:
        # Select all neighbors of the X_node
        neighbordhood = X_node.neighbors()

        for Y_node_i in neighbordhood:
            for Y_node_j in neighbordhood:
                # Ceck if both nodes are the same
                if Y_node_i['name'] != Y_node_j['name']:
                    # If there is no an edge generate
                    if not g.has_edge(Y_node_i['name'], Y_node_j['name']):
                        weight_ = get_edge_weight_2(Y_node_i, Y_node_j, biparte_network)
                        # print("Peso: ", Y_node_i['name'], "-", Y_node_j['name'], " => ", weight_)
                        g.add_edge(Y_node_i["name"], Y_node_j["name"],
                                   weight=weight_)

    # Convert from networkX graph to igraph graph
    g = ig.Graph.from_networkx(g)
    g.vs["name"] = g.vs["_nx_name"]
    del g.vs["_nx_name"]

    for u_nodes in g.vs:
        rsrcs = biparte_network.vs.find(name=u_nodes["name"]).neighbors()
        rsrcs = [r_node["name"] for r_node in rsrcs]
        u_nodes["rsrcs"] = rsrcs

    print("User Network builded!")
    print(g.summary())
    return g

In [47]:
### Projection of the networks

pos_user_user = bipartite_projection_2(pos_bip_network)
pos_res_res = bipartite_projection_rs(pos_bip_network)

neg_user_user = bipartite_projection_2(neg_bip_network)
neg_res_res = bipartite_projection_rs(pos_bip_network)


User Network builded!
IGRAPH UNW- 116 6219 -- 
+ attr: name (v), rsrcs (v), weight (e)
User Network builded!
IGRAPH UNW- 810 322639 -- 
+ attr: name (v), rsrcs (v), weight (e)
User Network builded!
IGRAPH UNW- 108 5365 -- 
+ attr: name (v), rsrcs (v), weight (e)
User Network builded!
IGRAPH UNW- 810 322639 -- 
+ attr: name (v), rsrcs (v), weight (e)


# Detección Comunidades

## Positive

### Users

In [48]:
### Community detection algorithm

# Louvain Algorithm
parti_pos_user_user = pos_user_user.community_multilevel(
    weights=pos_user_user.es()["weight"])
print("Modularity: %.4f" % parti_pos_user_user.modularity) # Modualarity value

# Add cluster attribute to nodes
pos_user_user.vs["commty"] = parti_pos_user_user.membership

print(pos_user_user.summary())
print(parti_pos_user_user.summary())

dict_commts = sub_community_detection(pos_user_user, 0.5, None)
len(dict_commts)
# Obtener el máximo valor de recursos en el total de comunidades
n_res_in_comms = [len(i[1]) for i in dict_commts.values()]
max_n_res = max(n_res_in_comms)
print("Comunidad con # mayor recursos", max_n_res)

# Umbrales para la clasificación de comunidades
big_threshold = int(0.50 * max_n_res)
med_threshold = int(0.25 * max_n_res)
print("Big Threshold: ", big_threshold, " \t\t Med Threshold", med_threshold)

Modularity: 0.0095
IGRAPH UNW- 116 6219 -- 
+ attr: commty (v), name (v), rsrcs (v), weight (e)
Clustering with 116 elements and 3 clusters
Comunidad con # mayor recursos 804
Big Threshold:  402  		 Med Threshold 201


### Resourcces

In [49]:
### Community detection algorithm

# Louvain Algorithm
parti_pos_res_res = pos_res_res.community_multilevel(
    weights=pos_res_res.es()["weight"])
print("Modularity: %.4f" % parti_pos_res_res.modularity) # Modualarity value

# Add cluster attribute to nodes
pos_res_res.vs["commty"] = parti_pos_res_res.membership

print(pos_res_res.summary())
print(parti_pos_res_res.summary())

dict_commts = sub_community_detection(pos_res_res, 0.5, None)
len(dict_commts)
# Obtener el máximo valor de recursos en el total de comunidades
n_res_in_comms = [len(i[1]) for i in dict_commts.values()]
max_n_res = max(n_res_in_comms)
print("Comunidad con # mayor recursos", max_n_res)

# Umbrales para la clasificación de comunidades
big_threshold = int(0.50 * max_n_res)
med_threshold = int(0.25 * max_n_res)
print("Big Threshold: ", big_threshold, " \t\t Med Threshold", med_threshold)

Modularity: 0.0092
IGRAPH UNW- 810 322639 -- 
+ attr: commty (v), name (v), rsrcs (v), weight (e)
Clustering with 810 elements and 2 clusters
Comunidad con # mayor recursos 112
Big Threshold:  56  		 Med Threshold 28


## Negative

### Users

In [50]:
### Community detection algorithm

# Louvain Algorithm
parti_neg_user_user = neg_user_user.community_multilevel(
    weights=neg_user_user.es()["weight"])
print("Modularity: %.4f" % parti_neg_user_user.modularity) # Modualarity value

# Add cluster attribute to nodes
neg_user_user.vs["commty"] = parti_neg_user_user.membership

print(neg_user_user.summary())
print(parti_neg_user_user.summary())

dict_commts = sub_community_detection(neg_user_user, 0.5, None)
len(dict_commts)
# Obtener el máximo valor de recursos en el total de comunidades
n_res_in_comms = [len(i[1]) for i in dict_commts.values()]
max_n_res = max(n_res_in_comms)
print("Comunidad con # mayor recursos", max_n_res)

# Umbrales para la clasificación de comunidades
big_threshold = int(0.50 * max_n_res)
med_threshold = int(0.25 * max_n_res)
print("Big Threshold: ", big_threshold, " \t\t Med Threshold", med_threshold)

Modularity: 0.0063
IGRAPH UNW- 108 5365 -- 
+ attr: commty (v), name (v), rsrcs (v), weight (e)
Clustering with 108 elements and 2 clusters
Comunidad con # mayor recursos 768
Big Threshold:  384  		 Med Threshold 192


### Resources

In [51]:
### Community detection algorithm

# Louvain Algorithm
parti_neg_res_res = neg_res_res.community_multilevel(
    weights=neg_res_res.es()["weight"])
print("Modularity: %.4f" % parti_neg_res_res.modularity) # Modualarity value

# Add cluster attribute to nodes
neg_res_res.vs["commty"] = parti_neg_res_res.membership

print(neg_res_res.summary())
print(parti_neg_res_res.summary())

dict_commts = sub_community_detection(neg_res_res, 0.5, None)
len(dict_commts)
# Obtener el máximo valor de recursos en el total de comunidades
n_res_in_comms = [len(i[1]) for i in dict_commts.values()]
max_n_res = max(n_res_in_comms)
print("Comunidad con # mayor recursos", max_n_res)

# Umbrales para la clasificación de comunidades
big_threshold = int(0.50 * max_n_res)
med_threshold = int(0.25 * max_n_res)
print("Big Threshold: ", big_threshold, " \t\t Med Threshold", med_threshold)

Modularity: 0.0092
IGRAPH UNW- 810 322639 -- 
+ attr: commty (v), name (v), rsrcs (v), weight (e)
Clustering with 810 elements and 2 clusters
Comunidad con # mayor recursos 112
Big Threshold:  56  		 Med Threshold 28


# Main

## Users

In [52]:
users_nodes_in_both = set(pos_bip_network.vs.select(type=0)["name"]).intersection(
    set(neg_bip_network.vs.select(type=0)["name"]))
print("Number of resources in both bipartite network:", len(users_nodes_in_both))

contador = 0

# Recorrer por cada recurso y obtener los nodos vecinos en cada grafo bipartito.
for res in users_nodes_in_both:
    
    # Get the nodes in each community
    neighs_pos_node = []
    for commty in parti_pos_res_res.subgraphs(): # For each commty in the partition
        if search_resource_in_commty(res, commty):
            neighs_pos_node += [i["name"] for i in commty.vs()]            
    # Convert to set
    neighs_pos_node = set(neighs_pos_node)
    #print(neighs_pos_node)

    neighs_neg_node = []
    for commty in parti_neg_res_res.subgraphs(): # For each commty in the partition
        if search_resource_in_commty(res, commty):
            neighs_neg_node += [i["name"] for i in commty.vs()]
    # Convert to set
    neighs_neg_node = set(neighs_neg_node)
    

    # Calcular su intersección. Si es conjunto vacío está perfecto
    if len(neighs_pos_node.intersection(neighs_neg_node)) != 0: # Empty set
        contador += 1
        # print("Problem", res)
        # print(neighs_pos_node)
        # print(neighs_neg_node)
        # print(neighs_pos_node.intersection(neighs_neg_node))

print("Done")
print("Ruido =", contador)

Number of resources in both bipartite network: 107
Done
Ruido = 80


## Resources

In [53]:
res_nodes_in_both = set(pos_bip_network.vs.select(type=1)["name"]).intersection(
    set(neg_bip_network.vs.select(type=1)["name"]))
print("Number of resources in both bipartite network:", len(res_nodes_in_both))

contador = 0

# Recorrer por cada recurso y obtener los nodos vecinos en cada grafo bipartito.
for res in res_nodes_in_both:
    
    # Get the nodes in each community
    neighs_pos_node = []
    for commty in parti_pos_user_user.subgraphs(): # For each commty in the partition
        if search_resource_in_commty(res, commty):
            neighs_pos_node += [i["name"] for i in commty.vs()]            
    # Convert to set
    neighs_pos_node = set(neighs_pos_node)
    #print(neighs_pos_node)

    neighs_neg_node = []
    for commty in parti_neg_user_user.subgraphs(): # For each commty in the partition
        if search_resource_in_commty(res, commty):
            neighs_neg_node += [i["name"] for i in commty.vs()]
    # Convert to set
    neighs_neg_node = set(neighs_neg_node)
    

    # Calcular su intersección. Si es conjunto vacío está perfecto
    if len(neighs_pos_node.intersection(neighs_neg_node)) != 0: # Empty set
        contador += 1
        # print("Problem", res)
        # print(neighs_pos_node)
        # print(neighs_neg_node)
        # print(neighs_pos_node.intersection(neighs_neg_node))

print("Done")
print("Ruido =", contador)

Number of resources in both bipartite network: 777
Done
Ruido = 653
